In [1]:
import os
%pwd

'/Users/deryliankhalfani/Documents/Self_Learn/Udemy/MLOps Bootcamp/End2EndMLOpsBootcamp_NLP/research'

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size:  int
    weight_decay: float
    logging_steps:  int
    eval_strategy: str
    eval_steps: int
    save_steps:  int
    gradient_accumulation_steps:  int

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_path)
        self.params_filepath = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params_filepath.TrainingArguments

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,

        )

        return model_trainer_config


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
import torch
from  datasets import load_from_disk

/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-08-30 17:36:34,404: INFO: config: TensorFlow version 2.19.0 available. ]


In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):

        os.environ["DISABLE_MLFLOW_INTEGRATION"] = "1"
        os.environ["DISABLE_DAGSHUB_INTEGRATION"] = "1"

        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

        #loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir = 'pegasus-samsum', num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1000000,
            gradient_accumulation_steps=16
            )
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['test'], # use small data first
                  eval_dataset=dataset_samsum_pt['validation'],
                  callbacks = [])

        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, 'pegasus-samsum-model'))

        ## save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, 'tokenizer'))

In [8]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-08-30 17:36:34,670: INFO: common: yaml file: config/config.yaml loaded successfully ]
[2025-08-30 17:36:34,673: INFO: common: yaml file: params.yaml loaded successfully ]
[2025-08-30 17:36:34,674: INFO: common: Created directory at: artifacts ]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/1m/k2srhgm57x163x3p887yhdzr0000gn/T/ipykernel_13400/3678770525.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is se

RuntimeError: MPS backend out of memory (MPS allocated: 6.20 GiB, other allocations: 2.57 GiB, max allowed: 9.07 GiB). Tried to allocate 375.40 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).